<a href="https://colab.research.google.com/github/pumplay01/pumplay01/blob/main/TopicModeling_NTV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!pip install pyLDAvis

In [ ]:
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
!pip install --upgrade pythainlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 25.2 MB/s eta 0:00:00


In [ ]:
import pythainlp

In [ ]:
df = pd.read_csv('/content/Reviews.csv')

In [ ]:
df.tail()

,Review ID,Review
5,6,เขาดูถูกเรา วันนั้นเราแต่งตัวไม่ดี โทรศัพท์เรา...
6,7,รักษาดี วินิจฉัยตรงจุด ป่วยไปแอดมิดรพ.อื่นมา10...
7,8,ปกติพาลูกมาที่นี่ตลอด แต่รอบนี้เจอหมอที่รู้สึก...
8,9,ดีกว่าที่คาด มาหาหมอกระดูกวันอาทิตย์ เข้ามาคนเ...
9,10,ห้องพักดี สะดวกใกล้ห้างมาก แต่การรักษา ได้ยาแร...


**TOKENIZE**

In [ ]:
stopwords = list(pythainlp.corpus.thai_stopwords())

In [ ]:
removed_words = ['',' ','  ','\n','โรงพยาบาล', '(', ')','รพ','รพ.']

In [ ]:
screening_words = stopwords + removed_words

In [ ]:
def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ',' + word
  return merged[1:]

In [ ]:
df['Review_token'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [ ]:
df.head()

,Review ID,Review,Review_token,topics,score
0,1,โรงพยาบาลนนทเวช อยู่บนถนนงามวงศ์วาน ติดกับห้าง...,"นนทเวช,ถนน,งาม,วงศ์วาน,ติดกับ,ห้างสรรพสินค้า,เ...",3,0.998690
1,2,Serviceห่วยมาก มา follow ตามนัด ถึงก่อน15นาทีต...,"Service,ห่วย,follow,นัด,15,นาที,แนะนำ,ตรวจ,ตรง...",3,0.998477
2,3,รพ.แย่มาก ให้บริการช้ามากๆ ไม่เหมาะสมกับราคา ป...,"แย่มาก,ให้บริการ,เหมาะสม,ราคา,ป่วยหนัก,สนใจ,ปล...",3,0.995935
3,4,รู้สึกเสียใจมากที่ตัดสินใจไปรพ.นี้ เข้าใจว่าหม...,"รู้สึก,เสียใจ,ตัดสินใจ,หมอ,แต่ละคน,นิสัย,เหมือ...",3,0.998322
4,5,ผิดหวังกับเเผนกเด็กมาก คุณหมอไม่ค่อยใส่ใจ เเทน...,"ผิดหวัง,เ,เผ,นก,เด็ก,คุณหมอ,ใส่ใจ,เ,เท,น,ใช้เว...",0,0.996117


In [ ]:
df.tail()

,Review ID,Review,Review_token,topics,score
5,6,เขาดูถูกเรา วันนั้นเราแต่งตัวไม่ดี โทรศัพท์เรา...,"ดูถูก, ,แต่งตัว,ดี, ,โทรศัพท์,แบต, ,ติดต่อ,ประ...",4,0.995322
6,7,รักษาดี วินิจฉัยตรงจุด ป่วยไปแอดมิดรพ.อื่นมา10...,"รักษา,ดี, ,วินิจฉัย,ตรงจุด, ,ป่วย,แอด,มิด,10,ห...",4,0.995322
7,8,ปกติพาลูกมาที่นี่ตลอด แต่รอบนี้เจอหมอที่รู้สึก...,"ปกติ,ลูก,ที่นี่, ,รอบ,เจอ,หมอ,รู้สึก,หมอ,เหมือ...",2,0.997579
8,9,ดีกว่าที่คาด มาหาหมอกระดูกวันอาทิตย์ เข้ามาคนเ...,"ดีกว่า,คาด, ,มาหา,หมอกระดูก,วันอาทิตย์, ,เข้าม...",1,0.995092
9,10,ห้องพักดี สะดวกใกล้ห้างมาก แต่การรักษา ได้ยาแร...,"ห้องพัก,ดี, ,สะดวก,ห้าง, ,การรักษา, ,ยา,แรง,เก...",4,0.996670


**CREATE DICTIONNARY**

In [ ]:
documents = df['Review_token'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [ ]:
print(dictionary.token2id.keys())

dict_keys([' ', 'การจราจร', 'คน', 'ค่าใช้จ่าย', 'งาม', 'ดี', 'ติดกัน', 'ติดกับ', 'ติดขัด', 'ถนน', 'ทั่วไป', 'ที่จอดรถ', 'นนทเวช', 'บริการ', 'บริเวณ', 'บุคลากร', 'ประตู', 'มอลล์', 'รอ', 'รองรับ', 'วงศ์วาน', 'สถานที่', 'สะอาด', 'สิทธิ์', 'หลากหลาย', 'ห้าง', 'ห้างสรรพสินค้า', 'เดอะ', 'เดิน', 'เดินทาง', 'เวลา', 'เอกชน', 'ใช้เวลา', '15', '45', '?????', 'Service', 'follow', 'กินข้าว', 'คนไข้', 'ชม.', 'ตรงเวลา', 'ตรวจ', 'ตัด', 'นัด', 'นั่ง', 'นาที', 'นึง', 'บ่ายโมง', 'รอบ', 'วน', 'หมอ', 'ห่วย', 'เดินเล่น', 'เหมือน', 'แนะนำ', 'แล้วก็', 'ครั้งสุดท้าย', 'ปล่อย', 'ป่วยหนัก', 'ราคา', 'สนใจ', 'เหมาะสม', 'แย่มาก', 'ให้บริการ', '25', '66', 'overtreat', 'การรักษา', 'คนดี', 'คืนนี้', 'ฉุกเฉิน', 'ตัดสินใจ', 'ตัวเอง', 'ต่อไป', 'ที่นี่', 'นิสัย', 'ปลอดภัย', 'ย.', 'รู้สึก', 'ห้อง', 'เสียใจ', 'เหมือนกัน', 'แต่ละคน', 'โดน', 'ไปหา', '  ', 'กลับมา', 'ข้อมูล', 'คุณหมอ', 'ซัก', 'ทาน', 'น', 'นก', 'บ', 'บาย', 'ผิดหวัง', 'ยย', 'ละเอียด', 'สาเหตุ', 'หา', 'เ', 'เข้าไป', 'เด็ก', 'เท', 'เป็นอันขาด', 'เผ', 'เเปป', 'ใข้'

In [ ]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

**TOPIC MODELING**

In [ ]:
num_topics = 4
chunksize = 4000
passes = 20
iterations = 50
eval_every = 1

temp = dictionary[0]
id2word = dictionary.id2token

model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                              alpha='auto', eta='auto', \
                              iterations=iterations, num_topics=num_topics, \
                              passes=passes, eval_every=eval_every)

In [ ]:
vis = pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary, mds='mmds')
vis

/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.082633 -0.039235       1        1  71.030848
1     -0.041090  0.095674       2        1  19.904229
0      0.076396 -0.062182       3        1   9.029643
2      0.047327  0.005743       4        1   0.035280, topic_info=         Term       Freq      Total Category  logprob  loglift
0              84.000000  84.000000  Default  30.0000  30.0000
42       ตรวจ   6.000000   6.000000  Default  29.0000  29.0000
51        หมอ   8.000000   8.000000  Default  28.0000  28.0000
54     เหมือน   4.000000   4.000000  Default  27.0000  27.0000
140       ปอด   4.000000   4.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
68   การรักษา   0.000716   2.794614   Topic4  -5.3608  -0.3201
175  เสียเงิน   0.000716   2.794622   Topic4  -5.3609  -0.3201
75     ที่นี่   0.000716   3.646343   Topic4  -5.3609  -0.5862
136       ค่า   0.000716   1.942888   Topic4  -5.3609   0.0434
46       นาที   0.000716   2.794627   Topic4  -5.3609  -0.3202

[197 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.825593         
0         2  0.153325         
0         3  0.023588         
86        1  0.460448         
86        3  0.460448         
...     ...       ...      ...
32        1  0.571872  ใช้เวลา
32        3  0.571872  ใช้เวลา
109       3  1.114939    ใส่ใจ
110       3  1.114890    ให้ยา
212       2  0.916391  ไม่ต้อง

[128 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 1, 3])

**PREDICT TOPICS**

In [ ]:
model.show_topic(1)

[(' ', 0.14541128),
 ('หมอ', 0.030517597),
 ('แอด', 0.023339015),
 ('มิด', 0.023338191),
 ('ดี', 0.023337599),
 ('ดู', 0.023337439),
 ('อาการ', 0.01615722),
 ('นัด', 0.01615711),
 ('รอ', 0.01615697),
 ('ยา', 0.016156951)]

In [ ]:
df['topics'] = df['Review_token'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_token'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [ ]:
print(df)

   Review ID                                             Review  \
0          1  โรงพยาบาลนนทเวช อยู่บนถนนงามวงศ์วาน ติดกับห้าง...   
1          2  Serviceห่วยมาก มา follow ตามนัด ถึงก่อน15นาทีต...   
2          3  รพ.แย่มาก ให้บริการช้ามากๆ ไม่เหมาะสมกับราคา ป...   
3          4  รู้สึกเสียใจมากที่ตัดสินใจไปรพ.นี้ เข้าใจว่าหม...   
4          5  ผิดหวังกับเเผนกเด็กมาก คุณหมอไม่ค่อยใส่ใจ เเทน...   
5          6  เขาดูถูกเรา วันนั้นเราแต่งตัวไม่ดี โทรศัพท์เรา...   
6          7  รักษาดี วินิจฉัยตรงจุด ป่วยไปแอดมิดรพ.อื่นมา10...   
7          8  ปกติพาลูกมาที่นี่ตลอด แต่รอบนี้เจอหมอที่รู้สึก...   
8          9  ดีกว่าที่คาด มาหาหมอกระดูกวันอาทิตย์ เข้ามาคนเ...   
9         10  ห้องพักดี สะดวกใกล้ห้างมาก แต่การรักษา ได้ยาแร...   

                                        Review_token  topics     score  
0  นนทเวช, ,ถนน,งาม,วงศ์วาน, ,ติดกับ,ห้างสรรพสินค...       3  0.998690  
1  Service,ห่วย, , ,follow, ,นัด, ,15,นาที,แนะนำ,...       3  0.998477  
2  แย่มาก, ,ให้บริการ, ,เหมาะสม,ราคา, ,ป่วย

In [ ]:
df.head()

,Review ID,Review,Review_token,topics,score
0,1,โรงพยาบาลนนทเวช อยู่บนถนนงามวงศ์วาน ติดกับห้าง...,"นนทเวช, ,ถนน,งาม,วงศ์วาน, ,ติดกับ,ห้างสรรพสินค...",3,0.998690
1,2,Serviceห่วยมาก มา follow ตามนัด ถึงก่อน15นาทีต...,"Service,ห่วย, , ,follow, ,นัด, ,15,นาที,แนะนำ,...",3,0.998477
2,3,รพ.แย่มาก ให้บริการช้ามากๆ ไม่เหมาะสมกับราคา ป...,"แย่มาก, ,ให้บริการ, ,เหมาะสม,ราคา, ,ป่วยหนัก,ส...",3,0.995935
3,4,รู้สึกเสียใจมากที่ตัดสินใจไปรพ.นี้ เข้าใจว่าหม...,"รู้สึก,เสียใจ,ตัดสินใจ, ,หมอ,แต่ละคน,นิสัย,เหม...",3,0.998322
4,5,ผิดหวังกับเเผนกเด็กมาก คุณหมอไม่ค่อยใส่ใจ เเทน...,"ผิดหวัง,เ,เผ,นก,เด็ก, ,คุณหมอ,ใส่ใจ, ,เ,เท,น,ใ...",0,0.996117


In [ ]:
df.tail()

,Review ID,Review,Review_token,topics,score
5,6,เขาดูถูกเรา วันนั้นเราแต่งตัวไม่ดี โทรศัพท์เรา...,"ดูถูก, ,แต่งตัว,ดี, ,โทรศัพท์,แบต, ,ติดต่อ,ประ...",3,0.998364
6,7,รักษาดี วินิจฉัยตรงจุด ป่วยไปแอดมิดรพ.อื่นมา10...,"รักษา,ดี, ,วินิจฉัย,ตรงจุด, ,ป่วย,แอด,มิด,10,ห...",3,0.998076
7,8,ปกติพาลูกมาที่นี่ตลอด แต่รอบนี้เจอหมอที่รู้สึก...,"ปกติ,ลูก,ที่นี่, ,รอบ,เจอ,หมอ,รู้สึก,หมอ,เหมือ...",3,0.999228
8,9,ดีกว่าที่คาด มาหาหมอกระดูกวันอาทิตย์ เข้ามาคนเ...,"ดีกว่า,คาด, ,มาหา,หมอกระดูก,วันอาทิตย์, ,เข้าม...",1,0.996665
9,10,ห้องพักดี สะดวกใกล้ห้างมาก แต่การรักษา ได้ยาแร...,"ห้องพัก,ดี, ,สะดวก,ห้าง, ,การรักษา, ,ยา,แรง,เก...",1,0.996961
